In [4]:
import numpy as np
from math import exp, log

# --- Input Parameters ---
S0 = 5.0      # Initial Stock Price
K = 10.0      # Strike Price
u = 2.0       # Up Factor
d = 0.5       # Down Factor
r = 0.2231    # Risk-free Rate (annual, continuously compounded)
Dt = 1.0      # Time Step (Delta t)
N = 50        # Number of Periods

# --- Derived Parameters ---
R = exp(r * Dt)           # Discrete Discount Factor (R = e^(r*Dt))
p = (R - d) / (u - d)     # Risk-Neutral Probability of Up Move
discount_factor = R**(-1) # Single-step Discount Factor

# Check for no-arbitrage condition (d < R < u)
if not (d < R < u):
    raise ValueError("Arbitrage opportunity exists: d < R < u condition violated.")

# --- 1. American Put Option Price (P_A) ---

# We use an array for option values, representing the prices at the current time step (n)
# The size is N+1, as the terminal step (n=N) has N+1 nodes.
P_American_values = np.zeros(N + 1)
S_prices = np.zeros(N + 1)

# a. Initialize Terminal Payoffs (n=N)
for j in range(N + 1):
    # Stock price S_T = S0 * u^j * d^(N-j)
    S_prices[j] = S0 * (u**j) * (d**(N - j))
    # Option value at expiry is the Intrinsic Value (IV)
    P_American_values[j] = max(K - S_prices[j], 0)

# b. Backward Induction (from n=N-1 down to n=0)
for n in range(N - 1, -1, -1):
    # Calculate stock prices for the current time step (n) only up to j=n
    # We only need to update the first n+1 elements of the array

    # 1. Continuation Value (CV) calculation
    # Vectorized calculation of CV using values from the next step (P_American_values)
    CV = (p * P_American_values[1:n+2] + (1 - p) * P_American_values[0:n+1]) * discount_factor
    
    # 2. Intrinsic Value (IV) and Early Exercise check
    for j in range(n + 1):
        # Calculate current stock price S_n,j
        stock_price = S0 * (u**j) * (d**(n - j))
        IV = max(K - stock_price, 0)
        
        # American Option Price = max(IV, CV)
        P_American_values[j] = max(IV, CV[j])

# The final American Put Price is the value at the root node (j=0)
P_American = P_American_values[0]

# --- 2. European Put Option Price (P_E) ---

# The European price calculation is the same as the American price, 
# but without the early exercise check (i.e., P_E = CV)

P_European_values = np.zeros(N + 1)

# a. Initialize Terminal Payoffs (n=N) - Same as American
for j in range(N + 1):
    S_prices[j] = S0 * (u**j) * (d**(N - j))
    P_European_values[j] = max(K - S_prices[j], 0)

# b. Backward Induction (n=N-1 down to n=0) - Only Continuation Value
for n in range(N - 1, -1, -1):
    # European Option Price = CV (Vectorized calculation)
    P_European_values[:n+1] = (p * P_European_values[1:n+2] + (1 - p) * P_European_values[0:n+1]) * discount_factor

P_European = P_European_values[0]

# --- 3. Early Exercise Premium (EEP) ---

EEP = P_American - P_European

# --- Output Results ---
print(f"--- 50-Period Binomial Tree Model Results (American Put) ---")
print(f"Parameters: S0=${S0:.2f}, K=${K:.2f}, r={r*100:.2f}%, N={N}")
print(f"Model Factors: u={u:.2f}, d={d:.2f}")
print(f"Risk-Neutral Probability (p): {p:.4f}")
print("-" * 55)
print(f"American Put Price (P_A): ${P_American:.8f}")
print(f"European Put Price (P_E): ${P_European:.8f}")
print(f"Early Exercise Premium (EEP = P_A - P_E): ${EEP:.8f}")
print("\n--- Analysis ---")
print(f"Initial Intrinsic Value (IV_0): ${max(K - S0, 0):.2f}")
print(f"Continuation Value at t=0 (CV_0): {P_European_values[0]:.8f}")
if P_American == max(K - S0, 0):
    print("Conclusion: Early exercise is optimal immediately (IV_0 > CV_0).")

--- 50-Period Binomial Tree Model Results (American Put) ---
Parameters: S0=$5.00, K=$10.00, r=22.31%, N=50
Model Factors: u=2.00, d=0.50
Risk-Neutral Probability (p): 0.5000
-------------------------------------------------------
American Put Price (P_A): $5.00000000
European Put Price (P_E): $0.00006908
Early Exercise Premium (EEP = P_A - P_E): $4.99993092

--- Analysis ---
Initial Intrinsic Value (IV_0): $5.00
Continuation Value at t=0 (CV_0): 0.00006908
Conclusion: Early exercise is optimal immediately (IV_0 > CV_0).
